In [1]:
import pykasso as pk
pk.create_project('test')

sks_settings = {
    'grid' : {
        'x0' : 0,
        'y0' : 0,
        'z0' : 0,
        'nx': 300,
        'ny': 300,
        'nz': 10,
        'dx': 10,
        'dy': 10,
        'dz': 10
    },
    'mask' : {
        # 'data' : 'test/inputs/betteraz_polygon.txt'
    },
    # 'topography' : {
    #     'data': 'test/inputs/betteraz_2D_z430.gslib',
    # },
    # 'geology' : {
    #     'data' : 'test/inputs/betteraz_2D_z430.gslib',
    #     'cost': {1: 0.5, 2: 0.7}
    # },
    # 'faults' : {
    #     'data' : 'test/inputs/faults.png',
    #     'axis': 'z',
    #     'cost': {1: 0.3, 2: 0.4}
    # },
    'fmm' : {
        'algorithm' : 'Riemann3'
    }
}

sim_settings = {
    'seed'    : 0,
    'outlets' : {
        'number'     : 4,
        'data'       : 'test/inputs/points.txt',
        'shuffle'    : False,
        'importance' : [1,1,1],
    },
    'inlets' : {
        'number'     : 4,
        'data'       : [],
        'shuffle'    : False,
        'per_outlet' : [1,1,1],
        'importance' : [1,1,1]
    },
}

CAUTION: You are using the development version of this package.


In [2]:
for _ in range(1):
    sim = pk.SKS(sks_settings=sks_settings, sim_settings=sim_settings)
    
    sim.build_model()

    print(sim.OUTLETS)
    
    # sim.compute_karst_networks()

[array([100., 100.,  50.]), array([200., 200.,  50.]), array([300., 300.,  50.]), array([400., 400.,  50.])]
[(2628.1359753952556, 1208.523799047211), (2871.506963387947, 1115.6633981209502), (2261.080797850388, 1573.269103150512), (2448.6778568318255, 907.8478597987818)]
       x      y     z
0  100.0  100.0  50.0
1  200.0  200.0  50.0
2  300.0  300.0  50.0
3  400.0  400.0  50.0


In [ ]:
# import numpy as np
# import pykasso.visualization as pkv

# settings = {
#     # 'show_grid' : True,
#     'ghost'   : [0],
#     'mask'    : True,
#     # 'slice'   : True
#     # 'inlets' : True,
#     # 'outlets' : True
# }
# pkv.show(sim, feature='geology', engine='pyvista', settings=settings)

In [ ]:
### ARRAY VISUALIZATION

# import numpy as np
# import pyvista as pv
# pv.global_theme.notebook = False

# data = sim.maps['karst'][-1]
# grid = sim.GRID

# mesh = pv.UniformGrid()
# mesh.dimensions = np.array((grid.nx, grid.ny, grid.nz)) + 1
# mesh.origin     = (grid.x0 - grid.dx/2, grid.y0 - grid.dy/2, grid.z0 - grid.dz/2)
# mesh.spacing    = (grid.dx, grid.dy, grid.dz)
# mesh            = mesh.flip_z(inplace=False)
# mesh            = mesh.cast_to_unstructured_grid()
# mesh.cell_data['data'] = data.flatten(order="F")

# ghosts = np.argwhere(mesh['data'] == 0)
# mesh = mesh.remove_cells(ghosts)

# plotter = pv.Plotter()
# _ = plotter.add_mesh(mesh)
# plotter.show(cpos='xy')


In [ ]:
# data = domain.TOPOGRAPHY.surface
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots()
# plt.imshow(data)
# plt.colorbar()

In [ ]:
# ### V Visualization
# import pyvista as pv
# pv.global_theme.notebook = False
# p = pv.Plotter()

# for (i, polygon) in enumerate(POLYGONS):
#     p.add_mesh(polygon, show_edges=True, opacity=0.5, color="w", lighting=False)

# for (i, points) in enumerate(POINTS):
#     intersection = pv.PolyData(points)
#     try:
#         p.add_mesh(intersection, color="maroon", point_size=25, label="Intersection Points")
#     except:
#         pass

# # for (i, ray) in enumerate(RAYS):
# #     p.add_mesh(ray, color="blue", line_width=5, label="Ray Segment n{}".format(i))

# p.show_bounds()
# # p.add_legend()
# p.show()

In [ ]:
# import pykasso.tools as pkt
# pkt.read_gslib('notebooks/test4/inputs/betteraz_2D_z430.gslib', 1)

In [ ]:
# def read_gslib(file):
#     import numpy as np
#     with open(file) as f:
#         data = f.readlines()
#     nvar = int(data[1])
#     lines_to_skip = nvar + 2
#     ndata = np.genfromtxt(file, skip_header=lines_to_skip)
#     return ndata

# read_gslib('notebooks/test4/inputs/betteraz_2D_z430.gslib')

In [ ]:
# import numpy as np
# rng = np.random.default_rng()
# surf = rng.poisson(600, size=(300,300))

# with open('test.gslib', mode='w') as f:
#     f.write('nx=300, ny=300 \n')
#     f.write('1 \n')
#     f.write('topography \n')
#     for i in range(300):
#         for j in range(300):
#             f.write(str(surf[i][j])+'\n')